# MDRS Pathfinding Demo

This notebook demonstrates how to use the `terrain_navigate` library to plan paths on MDRS terrain data.

## Prerequisites

Ensure you have run the following scripts:
1. `01_download_data.py` (to fetch metadata)
2. `02_process_data.py` (to generate the map file)

In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Add project root to path
project_root = Path("../../").resolve()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from src.terrain_navigate import AStar, GridEnvironment, PowerCost

## 1. Load Data

In [ ]:
data_path = Path("data/processed_map.npy")
if not data_path.exists():
    raise FileNotFoundError("Please run 02_process_data.py first!")

Z = np.load(data_path)
print(f"Loaded map with shape: {Z.shape}")

plt.figure(figsize=(8, 6))
plt.imshow(Z, cmap='terrain', origin='lower')
plt.colorbar(label='Elevation (m)')
plt.title("Terrain Map")
plt.show()

## 2. Setup Environment and Cost Function

In [ ]:
# Define environment bounds (matching the synthetic generation)
env = GridEnvironment(
    Z=Z,
    global_x_min=0.0,
    global_x_max=100.0,
    global_y_min=0.0,
    global_y_max=100.0,
    resolution=1.0
)

# Power cost function (penalizes slopes)
cost_fn = PowerCost(n=2.0, alpha=10.0)

## 3. Plan Path

In [ ]:
astar = AStar()

start = np.array([10.0, 10.0, 0.0])
goal = np.array([90.0, 90.0, 0.0])

# Snap to nearest nodes
start_node = env.get_nearest_node(start)
goal_node = env.get_nearest_node(goal)

print(f"Planning from {start_node} to {goal_node}...")

path, cost = astar.find_path(start_node, goal_node, env, cost_fn)

if path is None:
    print("No path found!")
else:
    print(f"Path found! Length: {len(path)} nodes, Cost: {cost:.2f}")

## 4. Visualize Result

In [ ]:
if path is not None:
    path_arr = np.array(path)
    
    plt.figure(figsize=(10, 8))
    plt.imshow(Z, cmap='terrain', origin='lower', extent=[0, 100, 0, 100])
    plt.colorbar(label='Elevation (m)')
    
    plt.plot(path_arr[:, 0], path_arr[:, 1], 'r-', linewidth=2, label='Path')
    plt.plot(start_node[0], start_node[1], 'go', markersize=10, label='Start')
    plt.plot(goal_node[0], goal_node[1], 'rx', markersize=10, label='Goal')
    
    plt.legend()
    plt.title("Planned Path on Terrain")
    plt.xlabel("X (m)")
    plt.ylabel("Y (m)")
    plt.grid(True, alpha=0.3)
    plt.show()